# Kiểm tra 

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [4]:
%%sql
select
    first_name,
    last_name
from customer
limit 5

Done.


first_name,last_name
Luís,Gonçalves
Leonie,Köhler
François,Tremblay
Bjørn,Hansen
František,Wichterlová


In [6]:
%%sql
with genre_sold_USA as(
    select g.name genre,
           sum(il.quantity) sum_sold
    from genre g
    left join track t on t.genre_id = g.genre_id
    left join invoice_line il on il.track_id = t.track_id
    left join invoice i on i.invoice_id = il.invoice_id
    left join customer c on c.customer_id = i.customer_id
    
    where country = 'USA'
    group by 1
    order by 2 DESC
)
select gsu.genre genre,
       gsu.sum_sold absolute numbers,
       Round(gsu.sum_sold * 100 / (select count(*) from genre_sold_USA gsu),2) Percentages_sold
from genre_sold_USA

(sqlite3.OperationalError) near "numbers": syntax error
[SQL: with genre_sold_USA as(
    select g.name genre,
           sum(il.quantity) sum_sold
    from genre g
    left join track t on t.genre_id = g.genre_id
    left join invoice_line il on il.track_id = t.track_id
    left join invoice i on i.invoice_id = il.invoice_id
    left join customer c on c.customer_id = i.customer_id
    
    where country = 'USA'
    group by 1
    order by 2 DESC
)
select gsu.genre genre,
       gsu.sum_sold absolute numbers,
       Round(gsu.sum_sold * 100 / (select count(*) from genre_sold_USA gsu),2) Percentages_sold
from genre_sold_USA]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [6]:
%%sql
select e.first_name || " " || e.last_name Employee_sales,
       e.title Title,
       e.birthdate Birthdate,
       e.hire_date Hire_date,
       round(sum(i.total),2) Total_sales_$,
       round(sum(i.total) * 100 / (select sum(total) from invoice),2) Per_sales
from employee e
left join customer c on c.support_rep_id = e.employee_id
left join invoice i on c.customer_id = i.customer_id
where e.title = "Sales Support Agent"
group by 1
order by 6 DESC;

Done.


Employee_sales,Title,Birthdate,Hire_date,Total_sales_$,Per_sales
Jane Peacock,Sales Support Agent,1973-08-29 00:00:00,2017-04-01 00:00:00,1731.51,36.77
Margaret Park,Sales Support Agent,1947-09-19 00:00:00,2017-05-03 00:00:00,1584.0,33.63
Steve Johnson,Sales Support Agent,1965-03-03 00:00:00,2017-10-17 00:00:00,1393.92,29.6


In [12]:
%%sql
with genre_sold_USA as(
    select g.name genre,
           sum(il.quantity) sum_sold
    from genre g
    left join track t on t.genre_id = g.genre_id
    left join invoice_line il on il.track_id = t.track_id
    left join invoice i on i.invoice_id = il.invoice_id
    left join customer c on c.customer_id = i.customer_id
    
    where c.country = 'USA'
    group by 1
    order by 2 DESC
)
Select gsu.genre Genre,
       gsu.sum_sold Absolute_sold,
       Round(gsu.sum_sold * 100 / (select total(gsu.sum_sold) from genre_sold_USA gsu),2) Percentage_sold
from genre_sold_USA gsu;
    
                                
                                                

Done.


Genre,Absolute_sold,Percentage_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


In [19]:
%%sql
with track_invoice as(
    select * from invoice i
    inner join invoice_line il on il.invoice_id = i.invoice_id
    inner join customer c on c.customer_id = i.customer_id
    where c.country = 'USA'
)
select g.name genre,
       count(tr.invoice_line_id) absolute_numbers,
       cast(count(tr.invoice_line_id) as float) / ( select count(*) from track_invoice) percentage_sold
from track_invoice tr
inner join track t on t.track_id = tr.track_id
inner join genre g on g.genre_id = t.genre_id
group by g.name
order by 2 DESC

Done.


genre,absolute_numbers,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


# Choose the new product for sale at stores.

In [8]:
%%sql
WITH track_invoice AS
(
    SELECT 
    *
    FROM invoice i 
    INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
)
SELECT
g.name genre,
COUNT(tr.invoice_line_id) genre_track_sold,
CAST(COUNT(tr.invoice_line_id) AS FLOAT) / (SELECT COUNT(*) FROM track_invoice) percentage_sold
FROM track_invoice tr 
INNER JOIN track t ON t.track_id = tr.track_id
INNER JOIN genre g ON t.genre_id = g.genre_id 
GROUP BY g.name
ORDER BY 2 DESC
LIMIT 30

Done.


genre,genre_track_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


 ## We can easily decide which albums to include according to their sales in the US.
## Ones to include: Punk,Blues,Pop

 ## Tìm xem nhân viên nào bán nhiều hàng nhất

In [26]:
%%sql
select e.first_name || " " || e.last_name Employee_name,
       e.title Title,
       e.birthdate Birthdate,
       max(invoice_date) latest_invoice_date,
       round(sum(i.total),2) Total_sales_$,
       round(sum(i.total) * 100 / (select sum(total) from invoice),2) Perc_sales
from employee e
left join customer c on c.support_rep_id = e.employee_id
left join invoice i on c.customer_id = i.customer_id
where e.title = "Sales Support Agent"
group by 1
order by 6 DESc;

Done.


Employee_name,Title,Birthdate,latest_invoice_date,Total_sales_$,Perc_sales
Jane Peacock,Sales Support Agent,1973-08-29 00:00:00,2020-12-30 00:00:00,1731.51,36.77
Margaret Park,Sales Support Agent,1947-09-19 00:00:00,2020-12-29 00:00:00,1584.0,33.63
Steve Johnson,Sales Support Agent,1965-03-03 00:00:00,2020-12-20 00:00:00,1393.92,29.6


In [16]:
%%sql
WITH track_invoice AS
(
    SELECT 
    *
    FROM invoice i 
    INNER JOIN invoice_line il ON i.invoice_id = il.invoice_id
    INNER JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
)
SELECT 
e.first_name || " " || e.last_name employee_name,
SUM(total) total_sale,
hire_date
FROM track_invoice tr
INNER JOIN employee e ON e.employee_id = tr.support_rep_id
GROUP BY employee_id
ORDER BY total_sale DESC
limit 20

Done.


employee_name,total_sale,hire_date
Margaret Park,5115.3300000000045,2017-05-03 00:00:00
Steve Johnson,3281.8500000000045,2017-10-17 00:00:00
Jane Peacock,2008.7100000000016,2017-04-01 00:00:00


 ## There are 3 employees who stand out with their sale.

In [17]:
%%sql
SELECT * FROM employee 
limit 5

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


## Analyze sale data based on different countries.

In [9]:
%%sql 
select sum(total) from invoice

Done.


sum(total)
4709.429999999994


In [7]:
%%sql
with track_invoice as(
    select *,
    case 
    when (select count(*) from customer group by country) = 1 then "Other"
    else c.country
    end as spe_cus
    from customer c
    inner join invoice i on c.customer_id = i.customer_id
    order by spe_cus
)
select country, round(sum(total),2) total_sale, count(customer_id) num_customer,
       round(cast(sum(total) as FLOAT) / count(customer_id),3) avg_cus,
       round(cast(sum(total) as FLOAT) / count(invoice_id),3) avg_invoice
from track_invoice tr
group by 1
order by 2 DESC;

Done.


country,total_sale,num_customer,avg_cus,avg_invoice
USA,1040.49,131,7.943,7.943
Canada,535.59,76,7.047,7.047
Brazil,427.68,61,7.011,7.011
France,389.07,50,7.781,7.781
Germany,334.62,41,8.161,8.161
Czech Republic,273.24,30,9.108,9.108
United Kingdom,245.52,28,8.769,8.769
Portugal,185.13,29,6.384,6.384
India,183.15,21,8.721,8.721
Ireland,114.84,13,8.834,8.834


In [18]:
%%sql
WITH track_invoice AS
(
   SELECT *,
    CASE 
    WHEN (SELECT COUNT(*) FROM customer GROUP BY country) = 1 THEN "Other"
    ELSE c.country
    END AS spe_cus
     
   FROM customer c
   INNER JOIN invoice i ON c.customer_id = i.customer_id
   ORDER BY spe_cus
)

SELECT
country,ROUND(sum(total),2) total_sale,COUNT(customer_id) num_customer,
ROUND(CAST(SUM(total) AS FLOAT)  / COUNT(customer_id),3) avg_cus,
ROUND(CAST(SUM(total) AS FLOAT)  / COUNT(invoice_id),3) avg_invoice
FROM track_invoice tr
GROUP BY 1
ORDER BY 2 DESC






Done.


country,total_sale,num_customer,avg_cus,avg_invoice
USA,1040.49,131,7.943,7.943
Canada,535.59,76,7.047,7.047
Brazil,427.68,61,7.011,7.011
France,389.07,50,7.781,7.781
Germany,334.62,41,8.161,8.161
Czech Republic,273.24,30,9.108,9.108
United Kingdom,245.52,28,8.769,8.769
Portugal,185.13,29,6.384,6.384
India,183.15,21,8.721,8.721
Ireland,114.84,13,8.834,8.834


In [19]:
%%sql
SELECT * FROM album 
limit 5

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [20]:
%%sql
SELECT * 
FROM invoice i
INNER JOIN invoice_line il ON il.invoice_id  = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN album al ON al.album_id = t.album_id
limit 5

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total,invoice_line_id,invoice_id_1,track_id,unit_price,quantity,track_id_1,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price_1,album_id_1,title,artist_id
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,1,1,1158,0.99,1,1158,Right Next Door to Hell,91,2,1,None,182321,3175950,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,2,1,1159,0.99,1,1159,Dust N' Bones,91,2,1,None,298374,5053742,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,3,1,1160,0.99,1,1160,Live and Let Die,91,2,1,None,184016,3203390,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,4,1,1161,0.99,1,1161,Don't Cry (Original),91,2,1,None,284744,4833259,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,5,1,1162,0.99,1,1162,Perfect Crime,91,2,1,None,143637,2550030,0.99,91,Use Your Illusion I,88


In [21]:
%%sql
SELECT * 
FROM invoice i
INNER JOIN invoice_line il ON il.invoice_id  = i.invoice_id
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN album al ON al.album_id = t.album_id
limit 50

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total,invoice_line_id,invoice_id_1,track_id,unit_price,quantity,track_id_1,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price_1,album_id_1,title,artist_id
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,1,1,1158,0.99,1,1158,Right Next Door to Hell,91,2,1,None,182321,3175950,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,2,1,1159,0.99,1,1159,Dust N' Bones,91,2,1,None,298374,5053742,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,3,1,1160,0.99,1,1160,Live and Let Die,91,2,1,None,184016,3203390,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,4,1,1161,0.99,1,1161,Don't Cry (Original),91,2,1,None,284744,4833259,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,5,1,1162,0.99,1,1162,Perfect Crime,91,2,1,None,143637,2550030,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,6,1,1163,0.99,1,1163,You Ain't the First,91,2,1,None,156268,2754414,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,7,1,1164,0.99,1,1164,Bad Obsession,91,2,1,None,328282,5537678,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,8,1,1165,0.99,1,1165,Back off Bitch,91,2,1,None,303436,5135662,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,9,1,1166,0.99,1,1166,Double Talkin' Jive,91,2,1,None,203637,3520862,0.99,91,Use Your Illusion I,88
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84,10,1,1167,0.99,1,1167,November Rain,91,2,1,None,537540,8923566,0.99,91,Use Your Illusion I,88


In [22]:
%%sql 
SELECT
COUNT(track_id),title,t.album_id
FROM album al 
INNER JOIN track t ON t.album_id = al.album_id
GROUP BY al.album_id
limit 5

Done.


COUNT(track_id),title,album_id
10,For Those About To Rock We Salute You,1
1,Balls to the Wall,2
3,Restless and Wild,3
8,Let There Be Rock,4
15,Big Ones,5


In [23]:
%%sql
WITH invoice_first_track  AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )
SELECT t.track_id ,t.name,t.album_id
FROM track t,invoice_first_track ifs
WHERE t.album_id = (
SELECT t2.album_id FROM track t2
WHERE t2.track_id = ifs.first_track_id)    
limit 50



Done.


track_id,name,album_id
1158,Right Next Door to Hell,91
1159,Dust N' Bones,91
1160,Live and Let Die,91
1161,Don't Cry (Original),91
1162,Perfect Crime,91
1163,You Ain't the First,91
1164,Bad Obsession,91
1165,Back off Bitch,91
1166,Double Talkin' Jive,91
1167,November Rain,91


In [26]:
%%sql
WITH invoice_first_track  AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )
SELECT t2.album_id FROM track t2,invoice_first_track ifs
WHERE t2.track_id = ifs.first_track_id
limit 50

Done.


album_id
91
20
203
58
163
5
6
9
20
215


In [37]:
%%sql
WITH invoice_first_track AS
(
    SELECT 
    il.invoice_id invoice_id,
    MIN(track_id) first_track_id
     FROM invoice_line il
    GROUP BY 1
)
SELECT album_purchase, COUNT(invoice_id) number_of_invoices,
CAST(COUNT(invoice_id) AS FLOAT) /(SELECT COUNT(*) FROM invoice) percent
FROM
(
SELECT ifs.*,    
CASE 
   WHEN
       (
           SELECT t.track_id FROM track t
           WHERE t.album_id = (
                             SELECT t2.album_id FROM track t2
                             WHERE t2.track_id = ifs.first_track_id
                            )
       EXCEPT 
           SELECT il2.track_id FROM invoice_line il2
           WHERE il2.invoice_id = ifs.invoice_id
       ) IS NULL
    AND
       (
           SELECT il2.track_id FROM invoice_line il2
           WHERE il2.invoice_id = ifs.invoice_id
        EXCEPT
           SELECT t.track_id FROM track t
           WHERE t.album_id = (
                             SELECT t.album_id FROM track t
                             WHERE t.track_id = ifs.first_track_id
                           )
           
       ) IS NULL
        Then "Yes"
        ELSE "No"
        END AS "album_purchase"
       
            

FROM invoice_first_track ifs
)    
GROUP BY album_purchase

Done.


album_purchase,number_of_invoices,percent
No,500,0.8143322475570033
Yes,114,0.18566775244299674
